In [1]:
# Import necessary libraries
import asyncio
import sys
import os
import warnings
from devtools import pprint
# Add the src directory to the Python path and suppress warnings
sys.path.append(os.path.abspath('../../../src/agent'))
warnings.filterwarnings("ignore", category=FutureWarning)
from market_intelligence_agent.config import TEAM_MEMBERS
from langchain_core.messages import convert_to_messages
from langgraph_sdk import get_client

import logging
import uuid


In [ ]:

# LangGraph Client
lg_client = get_client(url="http://localhost:2024")

user_query = ["What are some most recent news of Nvidia?"]
user_input_messages = user_query

thread = await lg_client.threads.create()

print("Starting graph stream...")
# NOTE: The input dictionary is simplified
async for chunk in lg_client.runs.stream(
    thread["thread_id"],
    assistant_id="market_intelligence_agent",
    input={
        # Constants
        "TEAM_MEMBERS": TEAM_MEMBERS,
        # Runtime Variables
        "messages": user_input_messages,
        # "mcp_tools": mcp_tools_global, # REMOVED
    },
    stream_mode="values"
):
    messages = chunk.data.get("messages", None)
    if messages:
        pprint(convert_to_messages(messages)[-1])
    print("=" * 50)

print("Graph stream finished.")



Starting graph stream...
HumanMessage(
    content='What are some most recent news of Nvidia?',
    additional_kwargs={},
    response_metadata={},
    id='0975f937-7063-4511-a38d-f9fc2a57456a',
)
HumanMessage(
    content=(
        '{\n'
        '  "thought": "The user wants to know the most recent news about Nvidia. I need to gather the latest news arti'
        'cles and summarize them.",\n'
        '  "title": "Gather and Summarize the Most Recent News about Nvidia",\n'
        '  "steps": [\n'
        '    {\n'
        '      "agent_name": "researcher",\n'
        '      "title": "Collect Recent News on Nvidia",\n'
        '      "description": "Search for the most recent news articles and updates about Nvidia from reputable source'
        's. Summarize the key points and developments."\n'
        '    },\n'
        '    {\n'
        '      "agent_name": "reporter",\n'
        '      "title": "Present a Summary Report of Nvidia\'s Recent News",\n'
        '      "description": "W

In [3]:
current_state = await lg_client.threads.get_state(thread["thread_id"])

In [4]:
current_state['values']['messages']

[{'content': 'What are some most recent news of Nvidia?',
  'additional_kwargs': {},
  'response_metadata': {},
  'type': 'human',
  'name': None,
  'id': '0975f937-7063-4511-a38d-f9fc2a57456a',
  'example': False},
 {'content': '{\n  "thought": "The user wants to know the most recent news about Nvidia. I need to gather the latest news articles and summarize them.",\n  "title": "Gather and Summarize the Most Recent News about Nvidia",\n  "steps": [\n    {\n      "agent_name": "researcher",\n      "title": "Collect Recent News on Nvidia",\n      "description": "Search for the most recent news articles and updates about Nvidia from reputable sources. Summarize the key points and developments."\n    },\n    {\n      "agent_name": "reporter",\n      "title": "Present a Summary Report of Nvidia\'s Recent News",\n      "description": "Write a professional summary report based on the researcher\'s findings, highlighting the most important and recent news about Nvidia."\n    }\n  ]\n}',
  'add

In [5]:
from IPython.display import Markdown, display

def display_markdown(content):
    start_marker = "<response>\n"
    end_marker = "</response>\n"
    start_index = content.find(start_marker)
    end_index = content.find(end_marker)
    if start_index != -1 and end_index != -1:
        content = content[start_index + len(start_marker):end_index]
    return Markdown(content)
    
    
# Display the formatted markdown
display(display_markdown(current_state['values']['messages'][-1]['content']))

# Executive Summary

This report provides an overview of the most recent news regarding Nvidia as of April 18, 2025. The information is based on reputable sources including Seeking Alpha, MarketWatch, The Motley Fool, LibertyRPF, and OfficeChai. Key topics include industry policy changes, analyst sentiment, competitive risks in China, and leadership insights from Nvidia’s CEO.

# Key Findings

- **Industry Policy Changes**: Nvidia is affected by tightening tariffs and modifications to the CHIPS Act, which may impact supply chains and costs.
- **Analyst Sentiment**: The company continues to receive strong support from analysts, with significant upside potential projected.
- **Competitive Landscape**: Nvidia remains a leading player in the semiconductor industry, with recent coverage highlighting its historical outperformance.
- **China Exposure**: Restrictions on chip sales to China present both risks and opportunities, potentially reducing long-term uncertainty but increasing competition from Chinese firms.
- **Leadership Perspective**: CEO Jensen Huang emphasizes the importance of resilience and learning from failure as keys to success.

# Detailed Analysis

## 1. Semiconductor Industry Updates: Tariffs and CHIPS Act Changes
- **Details**: Recent policy changes, including tighter tariffs and updates to the CHIPS Act, are expected to affect major chipmakers like Nvidia. These changes could lead to increased costs and supply chain adjustments.
- **Source**: Seeking Alpha (April 18, 2025)

## 2. Analyst Sentiment and Upside Potential
- **Details**: Nvidia is listed among 15 technology stocks with strong analyst backing and the potential for up to 85% growth within a year. The overall analyst sentiment remains bullish.
- **Source**: MarketWatch (April 18, 2025)

## 3. Industry Peer Updates: AMD, Nvidia, and TSMC
- **Details**: Industry updates focus on TSMC’s Q1 earnings and Nvidia’s continued relevance and outperformance in the semiconductor sector.
- **Source**: The Motley Fool (April 18, 2025)

## 4. Nvidia’s China Exposure and Long-Term Risks
- **Details**: Ongoing restrictions on selling certain chips (e.g., H20) to China may create short-term challenges for Nvidia. However, these restrictions could also reduce long-term uncertainty. There is a risk that Chinese competitors, such as Huawei, may benefit and erode Nvidia’s software advantages.
- **Source**: LibertyRPF (April 18, 2025)

## 5. Leadership Insights: Jensen Huang on Success
- **Details**: CEO Jensen Huang has publicly discussed the importance of developing a tolerance for failure as a critical factor in achieving success, reflecting on the entrepreneurial journey.
- **Source**: OfficeChai (April 18, 2025)

# Conclusions and Recommendations

## Conclusions
- Nvidia remains a strong performer in the technology sector, with continued analyst support and a reputation for innovation.
- The company faces potential headwinds from global policy changes and increased competition, particularly in China.
- Leadership continues to emphasize resilience and adaptability as core values.

## Recommendations
- **For Investors**: Monitor ongoing policy developments and Nvidia’s response to competitive pressures in China.
- **For Stakeholders**: Stay informed about supply chain adjustments and potential impacts from tariffs and CHIPS Act changes.
- **For Industry Observers**: Watch for further statements from Nvidia leadership regarding strategic direction and risk management.

---

**Sources**:  
- Seeking Alpha, MarketWatch, The Motley Fool, LibertyRPF, OfficeChai (all April 18, 2025)

*Information is based solely on the sources listed above. No additional data was used or assumed.*
